# Quickstart

## Working with data

PyTorch has two premitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset` stores the samples and their corresponding labels,and `DataLoader` wraps an iterable around the `Dataset`.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

print(f'Torch version: {torch.__version__}')

/Users/victor/dev/tutorials/torch-playground/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 2.0.0.dev20221213


The `torchvision.datasets` module contains `Dataset` objects for many real-world data like CIFAR, COC, etc. Here we use the FashionMNIST dataset. Every TorchVision `Dataset` includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

In [2]:
root = '../../../data/'
# Download traiing data from open datasets.
training_data = datasets.FashionMNIST(
    root=root,
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root=root,
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the `Dataset` as an argument to `DataLoader`. This wraps an iterable over our dataset, and supports automatic batching, sampling and multiprocess data loading. Here we define a batch size of 64, i.e. eaach element in the dataloader iterable will return a batch of 64 features and labels.

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

To define a neural network in PyTorch, we create a class that inherits from `nn.Module`. We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function. To accelerate operations in the neural network, we move it to the GPU if available.

In [4]:
# Get the CPU or GPU for training (mps - in case of Apple Silicon).
if torch.cuda.is_available():
    # Compute Unified Device Architecture.
    device = 'cuda'
elif torch.backends.mps.is_available():
    # Metal Performance Shader.
    device = 'mps'
else:
    # Central Processing Unit.
    device = 'cpu'

device = torch.device(device)
print(f'Using {device} device')

Using mps device


In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)
# compiled_model = torch.compile(model)
# print(compiled_model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

To train a model, we need a loss function and an optimizer.

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model's parameters.

In [7]:
def train(
    dataloader: DataLoader,
    model: nn.Module,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer
) -> None:
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error.
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'Loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

We also check the model's performance against the test dataset to ensure it is learning.

In [8]:
def test(
    dataloader: DataLoader,
    model: nn.Module,
    loss_fn: nn.Module,
) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # loss per batch(es).
    test_loss /= num_batches
    correct /= size

    print('Test error:')
    print(f'Accuracy: {correct:.02%}\tAvg loss: {test_loss:>8f}\n')

The training process is conducted over several iterations *(epochs)*. During each epoch, the model learns parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

In [9]:
epoch = 5
for t in range(1, epoch + 1):
    print(f'Epoch {t}\n-------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
Loss: 2.300339 [    0/60000]
Loss: 2.289175 [ 6400/60000]
Loss: 2.271161 [12800/60000]
Loss: 2.267830 [19200/60000]
Loss: 2.243491 [25600/60000]
Loss: 2.222013 [32000/60000]
Loss: 2.227760 [38400/60000]
Loss: 2.194563 [44800/60000]
Loss: 2.202615 [51200/60000]
Loss: 2.168777 [57600/60000]
Test error:
Accuracy: 46.89%	Avg loss: 2.162481

Epoch 2
-------------------------------
Loss: 2.169892 [    0/60000]
Loss: 2.164029 [ 6400/60000]
Loss: 2.108101 [12800/60000]
Loss: 2.124813 [19200/60000]
Loss: 2.079112 [25600/60000]
Loss: 2.016610 [32000/60000]
Loss: 2.046314 [38400/60000]
Loss: 1.968567 [44800/60000]
Loss: 1.981308 [51200/60000]
Loss: 1.909360 [57600/60000]
Test error:
Accuracy: 60.18%	Avg loss: 1.907112

Epoch 3
-------------------------------
Loss: 1.938391 [    0/60000]
Loss: 1.910949 [ 6400/60000]
Loss: 1.796414 [12800/60000]
Loss: 1.832676 [19200/60000]
Loss: 1.731495 [25600/60000]
Loss: 1.674767 [32000/60000]
Loss: 1.697982 [38400/60000]

## Saving Models

A common way to save a model is to serialize the internal state dictionary (containing model parameters).

In [10]:
saved_model = '../../../saved_models/intro'
model_path = f'{saved_model}/quickstart-model.pth'
# Create `save_model` path.
!mkdir -p {saved_model}

In [11]:
torch.save(model.state_dict(), model_path)
print(f'Saved PyTorch model state to {model_path}')

Saved PyTorch model state to ../../../saved_models/intro/quickstart-model.pth


## Loading Models

The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [12]:
model = NeuralNetwork()
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

This model can now be used to make predictions.

In [13]:
classes = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot',
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
